In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import stop_words
import nltk
from nltk.stem.porter import *
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from nltk.classify import NaiveBayesClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from collections import Counter

In [ ]:
# reading all data
xls = pd.ExcelFile('./Compiled_survey.xlsx')
Jan18 = pd.read_excel(xls, 'Jan 2018')
Dec17 = pd.read_excel(xls, 'Dec 2017')
Nov17 = pd.read_excel(xls, 'Nov 2017')
Oct17 = pd.read_excel(xls, 'Oct 2017')
Sept17 = pd.read_excel(xls, 'Sept 2017')
Aug17 = pd.read_excel(xls, 'August 2017')
Jul17 = pd.read_excel(xls, 'July 2017')
Jun17 = pd.read_excel(xls, 'June 2017')
May17 = pd.read_excel(xls, 'May 2017')
Apr17 = pd.read_excel(xls, 'April 2017')
Mar17 = pd.read_excel(xls, 'Mar 2017')
Feb17 = pd.read_excel(xls, 'Feb 2017')
Jan17 = pd.read_excel(xls, 'Jan 2017')

In [2]:
xls = pd.ExcelFile('./Compiled_survey.xlsx')
Jan18 = pd.read_excel(xls, 'Jan 2017')

In [ ]:
# data = Jan18.append([Dec17, Nov17, Oct17, Sept17, Aug17, Jul17, Jun17, May17, Apr17, Mar17, Feb17, Jan17])

In [3]:
my_additional_stop_words = ['know', 'like', 'just']

In [4]:
my_stop_words = stop_words.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

In [5]:
def tokenize(text):
    text = text.lower()
    text = re.sub('[' + string.punctuation + '0-9\\r\\t\\n]',' ',text)
    token = nltk.word_tokenize(text)
    token = [w for w in token if len(w )> 2]
    token = [w for w in token if not w in my_stop_words]
#     token1 = ' '.join(token)
    return token

def stemwords(words):
    stemmer = PorterStemmer()
    words = [w.decode('ascii', 'ignore') for w in words]
    stemmed = [stemmer.stem(w) for w in words]
    fwords = [w.encode('ascii', 'ignore') for w in stemmed]
    fwords1 = ' '.join(fwords)
    return fwords1

def tokenizer(text):
    return stemwords(tokenize(text))

### Cleaning a bit

In [6]:
# filtering for purspose, recommendation and level of effort
Jan18 = Jan18[(Jan18['What is the purpose for your visit today?'] != 'Account Update')&
      (Jan18['How likely is it that you would recommend FasTrak to a friend or colleague?'] != 10.0)&
      (Jan18['How did this effort compare to your expectations?'] != 'Exceptional')
     ]

In [7]:
# cleaning data
Jan18feedback = Jan18['Additional Feedback (Optional)']
Jan18feedback = Jan18feedback.dropna()
Jan18feedback = pd.DataFrame(Jan18feedback)
Jan18feedback = Jan18feedback[Jan18feedback['Additional Feedback (Optional)'] !='none']
Jan18feedback = Jan18feedback[Jan18feedback['Additional Feedback (Optional)'] !='n/a']
Jan18feedback["Additional Feedback (Optional)"] = Jan18feedback["Additional Feedback (Optional)"].str.encode('utf-8').astype(str)
Jan18feedback["tokenized"] = Jan18feedback["Additional Feedback (Optional)"].apply(tokenizer)

### Sentiment Analysis

In [8]:
list1 = Jan18feedback["Additional Feedback (Optional)"].unique()
analyzer = SentimentIntensityAnalyzer()

dict1 = {}
for i in list1:
    #print(i)
    dict1[i] = analyzer.polarity_scores(i)
    
Jan18feedback['compound'] = Jan18feedback["Additional Feedback (Optional)"].map(dict1)
survey = pd.concat([Jan18feedback.drop(['compound'], axis=1), Jan18feedback['compound'].apply(pd.Series)], axis=1)
# keeping data with high sentiment
survey_neg = survey[survey['pos'] < 0.7]  

### TFIDF

In [ ]:
tvec = TfidfVectorizer(tokenizer=tokenize, min_df=.0025, max_df=.1, stop_words='english', ngram_range=(1,4))

tvec_weights = tvec.fit_transform(survey_neg['tokenized'].dropna())
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(30)

In [ ]:
a = survey_neg[survey_neg['tokenized'].str.contains("payment")]

In [ ]:
a = survey_neg[(survey_neg['tokenized'].str.contains("abl"))&
           (survey_neg['tokenized'].str.contains("add"))]

In [ ]:
a.to_csv("/Users/danaiavg/Desktop/test1.csv")

In [ ]:
survey_neg.shape

### Topic Model